# Complete Workflow with PanelBox v0.8.0

**Tutorial Completo**: Test Runners & Master Report

Este notebook demonstra o workflow completo do PanelBox v0.8.0, incluindo:
1. Setup e preparação dos dados
2. PanelExperiment e ajuste de modelos
3. ValidationTest runner (NEW in v0.8.0!)
4. ComparisonTest runner (NEW in v0.8.0!)
5. Residual diagnostics
6. Master Report generation (NEW in v0.8.0!)
7. Temas e customização

---

## 1. Setup e Importações

Primeiro, vamos importar as bibliotecas necessárias e verificar a versão do PanelBox.

In [ ]:
import panelbox as pb
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print(f"PanelBox version: {pb.__version__}")
print(f"Expected: 0.8.0 or higher")

## 2. Carregar e Explorar Dados

Vamos usar o dataset Grunfeld de investimento, incluído no PanelBox.

In [ ]:
# Carregar dataset Grunfeld
data = pb.load_grunfeld()

print(f"Dataset shape: {data.shape}")
print(f"Panel structure: {data['firm'].nunique()} firms, {data['year'].nunique()} years")
print(f"\nVariables: {list(data.columns)}")

# Visualizar primeiras linhas
data.head(10)

### Estatísticas Descritivas

In [ ]:
data.describe()

## 3. Criar PanelExperiment

O **PanelExperiment** é a interface principal do PanelBox para gerenciar múltiplos modelos.

In [ ]:
# Criar experimento
experiment = pb.PanelExperiment(
    data=data,
    formula="invest ~ value + capital",
    entity_col="firm",
    time_col="year"
)

print(experiment)

## 4. Ajustar Múltiplos Modelos

Vamos ajustar 3 modelos diferentes: Pooled OLS, Fixed Effects e Random Effects.

In [ ]:
# Ajustar modelos individuais
experiment.fit_model('pooled_ols', name='ols')
experiment.fit_model('fixed_effects', name='fe')
experiment.fit_model('random_effects', name='re')

# Listar modelos ajustados
print(f"\nModelos ajustados: {experiment.list_models()}")
print(experiment)

### Resumo dos Modelos

In [ ]:
# Ver resumo de cada modelo
for model_name in experiment.list_models():
    results = experiment.get_model(model_name)
    print(f"\n{'='*60}")
    print(f"Model: {model_name.upper()}")
    print(f"{'='*60}")
    print(f"R²: {results.rsquared:.4f}")
    print(f"Adj. R²: {getattr(results, 'rsquared_adj', results.rsquared):.4f}")
    if hasattr(results, 'aic'):
        print(f"AIC: {results.aic:.2f}")
    if hasattr(results, 'bic'):
        print(f"BIC: {results.bic:.2f}")

## 5. ValidationTest Runner (NEW in v0.8.0!)

O **ValidationTest** runner permite rodar testes de validação com configs preset.

In [ ]:
from panelbox.experiment.tests import ValidationTest

# Criar runner
validation_runner = ValidationTest()

print(f"Configs disponíveis: {list(validation_runner.CONFIGS.keys())}")
print(f"\nQuick tests: {validation_runner.CONFIGS['quick']}")
print(f"Basic tests: {validation_runner.CONFIGS['basic']}")
print(f"Full tests: {validation_runner.CONFIGS['full']}")

### Rodar Validação com Config 'Full'

In [ ]:
# Validar modelo Fixed Effects
validation_result = experiment.validate_model('fe')

print(validation_result.summary())

### Gerar Validation Report HTML

In [ ]:
# Salvar report HTML
val_path = validation_result.save_html(
    'validation_report_v08.html',
    test_type='validation',
    theme='professional'
)

print(f"✅ Validation report saved to: {val_path}")

## 6. ComparisonTest Runner (NEW in v0.8.0!)

O **ComparisonTest** runner compara múltiplos modelos automaticamente.

In [ ]:
from panelbox.experiment.tests import ComparisonTest

# Criar runner
comparison_runner = ComparisonTest()
print(comparison_runner)

### Comparar Todos os Modelos

In [ ]:
# Comparar OLS, FE, RE
comparison_result = experiment.compare_models(['ols', 'fe', 're'])

print(comparison_result.summary())

### Identificar Melhor Modelo

In [ ]:
# Melhor modelo por AIC (menor é melhor)
best_aic = comparison_result.best_model('aic', prefer_lower=True)
print(f"Best model by AIC: {best_aic}")

# Melhor modelo por R² ajustado (maior é melhor)
best_r2 = comparison_result.best_model('rsquared_adj', prefer_lower=False)
print(f"Best model by Adj. R²: {best_r2}")

### Gerar Comparison Report HTML

In [ ]:
# Salvar comparison report
comp_path = comparison_result.save_html(
    'comparison_report_v08.html',
    test_type='comparison',
    theme='professional'
)

print(f"✅ Comparison report saved to: {comp_path}")

## 7. Residual Diagnostics (v0.7.0)

Analisar os resíduos do melhor modelo.

In [ ]:
# Analisar resíduos do modelo FE
residual_result = experiment.analyze_residuals('fe')

print(residual_result.summary())

### Testes Diagnósticos Individuais

In [ ]:
# Shapiro-Wilk (normalidade)
stat, pvalue = residual_result.shapiro_test
print(f"Shapiro-Wilk Test:")
print(f"  Statistic: {stat:.6f}")
print(f"  P-value: {pvalue:.6f}")
print(f"  Interpretation: {'Normal' if pvalue > 0.05 else 'Not normal'}")

# Durbin-Watson (autocorrelação)
dw = residual_result.durbin_watson
print(f"\nDurbin-Watson Test:")
print(f"  Statistic: {dw:.6f}")
print(f"  Interpretation: {'No autocorrelation' if 1.5 < dw < 2.5 else 'Autocorrelation present'}")

# Jarque-Bera (normalidade)
stat, pvalue = residual_result.jarque_bera
print(f"\nJarque-Bera Test:")
print(f"  Statistic: {stat:.6f}")
print(f"  P-value: {pvalue:.6f}")

# Ljung-Box (serial correlation)
stat, pvalue = residual_result.ljung_box
print(f"\nLjung-Box Test (10 lags):")
print(f"  Statistic: {stat:.6f}")
print(f"  P-value: {pvalue:.6f}")

### Gerar Residuals Report HTML

In [ ]:
# Salvar residuals report
res_path = residual_result.save_html(
    'residuals_report_v08.html',
    test_type='residuals',
    theme='professional'
)

print(f"✅ Residuals report saved to: {res_path}")

## 8. Master Report Generation (NEW in v0.8.0!)

O **Master Report** é um relatório abrangente que integra todos os sub-reports.

In [ ]:
# Gerar master report
master_path = experiment.save_master_report(
    'master_report_v08.html',
    theme='professional',
    title='PanelBox v0.8.0 - Complete Analysis',
    reports=[
        {
            'type': 'validation',
            'title': 'Fixed Effects Validation',
            'description': 'Specification tests for FE model',
            'file_path': 'validation_report_v08.html'
        },
        {
            'type': 'comparison',
            'title': 'Model Comparison',
            'description': 'Compare Pooled OLS, FE, and RE models',
            'file_path': 'comparison_report_v08.html'
        },
        {
            'type': 'residuals',
            'title': 'Residual Diagnostics',
            'description': 'Diagnostic tests for FE model residuals',
            'file_path': 'residuals_report_v08.html'
        }
    ]
)

print(f"✅ Master report saved to: {master_path}")
print(f"\nOpen this file in your browser to see:")
print(f"  - Experiment overview")
print(f"  - Summary of all fitted models")
print(f"  - Navigation to validation, comparison, and residuals reports")

## 9. Explorando Diferentes Temas

PanelBox oferece 3 temas profissionais: **professional**, **academic**, **presentation**.

### Theme: Academic

In [ ]:
# Gerar report com tema acadêmico
validation_result.save_html(
    'validation_academic.html',
    test_type='validation',
    theme='academic',
    title='Validation Report - Academic Style'
)

print("✅ Academic theme report saved")

### Theme: Presentation

In [ ]:
# Gerar report com tema para apresentação
comparison_result.save_html(
    'comparison_presentation.html',
    test_type='comparison',
    theme='presentation',
    title='Model Comparison - Presentation Style'
)

print("✅ Presentation theme report saved")

## 10. Exportar para JSON

Todos os resultados podem ser exportados para JSON para análise programática.

In [ ]:
# Exportar todos os resultados para JSON
validation_result.save_json('validation_v08.json')
comparison_result.save_json('comparison_v08.json')
residual_result.save_json('residuals_v08.json')

print("✅ All results exported to JSON")
print("  - validation_v08.json")
print("  - comparison_v08.json")
print("  - residuals_v08.json")

## 11. Workflow Completo Resumido

Aqui está todo o workflow em um único bloco:

In [ ]:
# Workflow completo em ~10 linhas
import panelbox as pb

# 1. Load data and create experiment
data = pb.load_grunfeld()
experiment = pb.PanelExperiment(data, "invest ~ value + capital", "firm", "year")

# 2. Fit models
experiment.fit_all_models(names=['pooled', 'fe', 're'])

# 3. Generate all reports
experiment.validate_model('fe').save_html('val.html', test_type='validation')
experiment.compare_models(['pooled', 'fe', 're']).save_html('comp.html', test_type='comparison')
experiment.analyze_residuals('fe').save_html('res.html', test_type='residuals')

# 4. Generate master report
experiment.save_master_report('master.html', reports=[
    {'type': 'validation', 'title': 'Validation', 'file_path': 'val.html'},
    {'type': 'comparison', 'title': 'Comparison', 'file_path': 'comp.html'},
    {'type': 'residuals', 'title': 'Residuals', 'file_path': 'res.html'}
])

print("✅ Complete workflow executed!")

## 12. Resumo das Novidades v0.8.0

### ✨ Test Runners
- **ValidationTest**: Runner configurável com presets (quick, basic, full)
- **ComparisonTest**: Comparação automática de múltiplos modelos
- APIs limpas e intuitivas

### ✨ Master Report
- Relatório mestre com overview do experimento
- Resumo de todos os modelos ajustados
- Navegação para validation, comparison e residuals reports
- Quick start guide embutido
- Design responsivo

### ✨ Workflow Integration
- Pipeline completo end-to-end
- Integração perfeita entre componentes
- Exportação JSON para todos os resultados
- 3 temas profissionais

---

## 📚 Próximos Passos

1. **Explore os HTML reports** gerados
2. **Experimente diferentes temas** (professional, academic, presentation)
3. **Use seus próprios dados** - substitua o dataset Grunfeld
4. **Customize os reports** - adicione título, descrições personalizadas
5. **Exporte para JSON** - análise programática dos resultados

---

**Made with ❤️ using PanelBox v0.8.0**